In [200]:
import pandas as pd
import glob
import numpy as np
import copy
import os

In [298]:
path = '../data/full/pl'
# path = '../data/full/championship'

files_list = [f for f in glob.glob(path + "/*.csv", recursive=True)]

matches = pd.DataFrame()

for f in files_list:
    fields = ['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST', 'WHH', 'WHD', 'WHA']
    curr_season = pd.read_csv(f, error_bad_lines=False, usecols=fields)
    curr_season.dropna(inplace=True)
    curr_season['FTHG'] = curr_season['FTHG'].astype(int)
    curr_season['FTAG'] = curr_season['FTAG'].astype(int)
    matches = matches.append(curr_season, ignore_index=True)

In [299]:
matches.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,WHH,WHD,WHA
0,Arsenal,Everton,2,1,H,11.0,13.0,5.0,7.0,1.28,4.5,8.00
1,Birmingham,Tottenham,1,0,H,10.0,15.0,5.0,7.0,2.20,3.2,2.80
2,Blackburn,Wolves,5,1,H,25.0,8.0,13.0,5.0,1.57,3.5,5.00
3,Fulham,Middlesbrough,3,2,H,17.0,8.0,9.0,5.0,2.30,3.2,2.62
4,Leicester,Southampton,2,2,D,12.0,13.0,7.0,10.0,2.30,3.2,2.62


In [300]:
matches.shape

(7226, 12)

In [301]:
last_n_games = 10

match = {}
team = {}
team_stats = {}

In [302]:
for i in range(len(matches)):
    match_index = len(matches) - i - 1
    curr_match = matches.iloc[len(matches) - i - 1]
    match[match_index] = [(curr_match['HomeTeam'], curr_match['AwayTeam']), [0] * 11, [0] * 11,
                          [curr_match['WHH'], curr_match['WHD'], curr_match['WHA']],
                          curr_match['FTR']]

    if curr_match['HomeTeam'] in team:
        team[curr_match['HomeTeam']] = team[curr_match['HomeTeam']] + [match_index]
    else:
        team[curr_match['HomeTeam']] = [match_index]

    if curr_match['AwayTeam'] in team:
        team[curr_match['AwayTeam']] = team[curr_match['AwayTeam']] + [match_index]
    else:
        team[curr_match['AwayTeam']] = [match_index]

    if curr_match['HomeTeam'] in team_stats:
        win = 0
        draw = 0
        loss = 0
        num_matches = 1
        if curr_match['FTR'] == 'H':
            win = 1
        elif curr_match['FTR'] == 'D':
            draw = 1
        else:
            loss = 1
        goals_scored = curr_match['FTHG']
        goals_conceded = curr_match['FTAG']
        team_stats[curr_match['HomeTeam']][0] = [sum(x) for x in zip(team_stats[curr_match['HomeTeam']][0],
                                                                     [win, draw, loss, goals_scored, goals_conceded,
                                                                      num_matches])]
    else:
        win = 0
        draw = 0
        loss = 0
        if curr_match['FTR'] == 'H':
            win = 1
        elif curr_match['FTR'] == 'D':
            draw = 1
        else:
            loss = 1
        team_stats[curr_match['HomeTeam']] = [[win, draw, loss, curr_match['FTHG'], curr_match['FTAG'], 1], [0] * 6]

    if curr_match['AwayTeam'] in team_stats:
        win = 0
        draw = 0
        loss = 0
        num_matches = 1
        if curr_match['FTR'] == 'A':
            win = 1
        elif curr_match['FTR'] == 'D':
            draw = 1
        else:
            loss = 1
        goals_scored = curr_match['FTAG']
        goals_conceded = curr_match['FTHG']
        team_stats[curr_match['AwayTeam']][1] = [sum(x) for x in zip(team_stats[curr_match['AwayTeam']][1],
                                                                     [win, draw, loss, goals_scored, goals_conceded,
                                                                      num_matches])]
    else:
        win = 0
        draw = 0
        loss = 0
        if curr_match['FTR'] == 'A':
            win = 1
        elif curr_match['FTR'] == 'D':
            draw = 1
        else:
            loss = 1
        team_stats[curr_match['AwayTeam']] = [[0] * 6, [win, draw, loss, curr_match['FTAG'], curr_match['FTHG'], 1]]

    if len(team[curr_match['HomeTeam']]) > last_n_games:
        curr_team = copy.deepcopy(curr_match['HomeTeam'])
        match_key = copy.deepcopy(team[curr_team].pop(0))
        curr_team_matches = copy.deepcopy(team[curr_team])

        games_total = 0
        wins_total = 0
        draws_total = 0
        losses_total = 0
        goals_scored_total = 0
        goals_conceded_total = 0

        shots_total = 0
        shots_on_target_total = 0
        shots_op = 0
        shots_on_target_op = 0

        for key in curr_team_matches:
            games_total += 1
            match_calc = matches.iloc[key]
            if match_calc['FTR'] == 'H' and match_calc['HomeTeam'] == curr_team:
                wins_total += 1
            elif match_calc['FTR'] == 'A' and match_calc['AwayTeam'] == curr_team:
                wins_total += 1

            if match_calc['FTR'] == 'H' and match_calc['AwayTeam'] == curr_team:
                losses_total += 1
            elif match_calc['FTR'] == 'A' and match_calc['HomeTeam'] == curr_team:
                losses_total += 1

            if match_calc['FTR'] == 'D':
                draws_total += 1

            if match_calc['HomeTeam'] == curr_team:
                goals_scored_total += match_calc['FTHG']
                goals_conceded_total += match_calc['FTAG']

                shots_total += match_calc['HS']
                shots_on_target_total += match_calc['HST']
                shots_op += match_calc['AS']
                shots_on_target_op += match_calc['AST']
            elif match_calc['AwayTeam'] == curr_team:
                goals_scored_total += match_calc['FTAG']
                goals_conceded_total += match_calc['FTHG']

                shots_total += match_calc['AS']
                shots_on_target_total += match_calc['AST']
                shots_op += match_calc['HS']
                shots_on_target_op += match_calc['HST']

        home_or_away = 2
        if matches.iloc[match_key]['HomeTeam'] == curr_team:
            home_or_away = 1
        match[match_key][home_or_away] = [games_total, wins_total, draws_total,
                                          losses_total, goals_scored_total, goals_conceded_total,
                                          shots_total, shots_on_target_total, shots_op, shots_on_target_op,
                                          curr_team]

    if len(team[curr_match['AwayTeam']]) > last_n_games:
        curr_team = copy.deepcopy(curr_match['AwayTeam'])
        match_key = copy.deepcopy(team[curr_team].pop(0))
        curr_team_matches = copy.deepcopy(team[curr_team])

        games_total = 0
        wins_total = 0
        draws_total = 0
        losses_total = 0
        goals_scored_total = 0
        goals_conceded_total = 0

        shots_total = 0
        shots_on_target_total = 0
        shots_op = 0
        shots_on_target_op = 0

        for key in curr_team_matches:
            games_total += 1
            match_calc = matches.iloc[key]

            if match_calc['FTR'] == 'H' and match_calc['HomeTeam'] == curr_team:
                wins_total += 1
            elif match_calc['FTR'] == 'A' and match_calc['AwayTeam'] == curr_team:
                wins_total += 1

            if match_calc['FTR'] == 'H' and match_calc['AwayTeam'] == curr_team:
                losses_total += 1
            elif match_calc['FTR'] == 'A' and match_calc['HomeTeam'] == curr_team:
                losses_total += 1

            if match_calc['FTR'] == 'D':
                draws_total += 1

            if match_calc['HomeTeam'] == curr_team:
                goals_scored_total += match_calc['FTHG']
                goals_conceded_total += match_calc['FTAG']

                shots_total += match_calc['HS']
                shots_on_target_total += match_calc['HST']
                shots_op += match_calc['AS']
                shots_on_target_op += match_calc['AST']
            elif match_calc['AwayTeam'] == curr_team:
                goals_scored_total += match_calc['FTAG']
                goals_conceded_total += match_calc['FTHG']

                shots_total += match_calc['AS']
                shots_on_target_total += match_calc['AST']
                shots_op += match_calc['HS']
                shots_on_target_op += match_calc['HST']

        home_or_away = 2
        if matches.iloc[match_key]['HomeTeam'] == curr_team:
            home_or_away = 1
        match[match_key][home_or_away] = [games_total, wins_total, draws_total,
                                          losses_total, goals_scored_total, goals_conceded_total,
                                          shots_total, shots_on_target_total, shots_op, shots_on_target_op,
                                          curr_team]

In [324]:
matches_nn_input = []
rows_to_drop = []
for key, value in match.items():
    if np.count_nonzero(match[key][1]) == 0 or np.count_nonzero(match[key][2]) == 0:
        rows_to_drop.append(key)
    else:
        matches_nn_input.append(match[key][1][1:] + match[key][2][1:] + match[key][3] + [match[key][4]])

In [325]:
all_cols = ['HTW', 'HTD', 'HTL', 'HTGS', 'HTGC', 'HTShots',
            'HTShotsTarget', 'HTOpShots', 'HTOpShotsTarget', 'Home',
            'ATW', 'ATD', 'ATL', 'ATGS', 'ATGC', 'ATShots', 
            'ATShotsTarget', 'ATOpShots', 'ATOpShotsTarget', 'Away',
            'HomeOdd', 'DrawOdd', 'AwayOdd', 'Result']

In [326]:
df = pd.DataFrame(matches_nn_input, columns=all_cols)

In [327]:
df.head()

,HTW,HTD,HTL,HTGS,HTGC,HTShots,HTShotsTarget,HTOpShots,HTOpShotsTarget,Home,ATW,ATD,ATL,ATGS,ATGC,ATShots,ATShotsTarget,ATOpShots,ATOpShotsTarget,Away,HomeOdd,DrawOdd,AwayOdd,Result
0,3,0,7,7,16,120.0,39.0,138.0,46.0,Watford,5,4,1,22,10,190.0,64.0,81.0,28.0,Man City,10.00,5.50,1.29,A
1,3,2,5,8,10,91.0,30.0,124.0,50.0,Swansea,1,2,7,6,15,101.0,32.0,144.0,37.0,West Brom,2.05,3.50,3.50,H
2,3,4,3,8,11,129.0,36.0,143.0,41.0,Southampton,1,3,6,8,16,110.0,29.0,134.0,46.0,Stoke,1.75,3.75,4.50,A
3,3,5,2,10,6,119.0,38.0,107.0,36.0,Man United,5,1,4,15,15,109.0,31.0,148.0,47.0,Crystal Palace,2.50,3.30,2.80,H
4,6,3,1,17,8,151.0,55.0,86.0,24.0,Liverpool,1,3,6,8,20,96.0,32.0,152.0,49.0,Middlesbrough,1.10,9.00,26.00,H


In [328]:
# df = df.drop(columns=['HomeOdd', 'DrawOdd', 'AwayOdd'])

In [329]:
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', True)

In [330]:
# Change team names to int identifiers
stacked = df[['Home', 'Away']].stack()
df[['Home', 'Away']] = pd.Series(stacked.factorize()[0], index=stacked.index).unstack()

In [331]:
df.head()

,HTW,HTD,HTL,HTGS,HTGC,HTShots,HTShotsTarget,HTOpShots,HTOpShotsTarget,Home,ATW,ATD,ATL,ATGS,ATGC,ATShots,ATShotsTarget,ATOpShots,ATOpShotsTarget,Away,HomeOdd,DrawOdd,AwayOdd,Result
0,3,0,7,7,16,120.0,39.0,138.0,46.0,0,5,4,1,22,10,190.0,64.0,81.0,28.0,1,10.00,5.50,1.29,A
1,3,2,5,8,10,91.0,30.0,124.0,50.0,2,1,2,7,6,15,101.0,32.0,144.0,37.0,3,2.05,3.50,3.50,H
2,3,4,3,8,11,129.0,36.0,143.0,41.0,4,1,3,6,8,16,110.0,29.0,134.0,46.0,5,1.75,3.75,4.50,A
3,3,5,2,10,6,119.0,38.0,107.0,36.0,6,5,1,4,15,15,109.0,31.0,148.0,47.0,7,2.50,3.30,2.80,H
4,6,3,1,17,8,151.0,55.0,86.0,24.0,8,1,3,6,8,20,96.0,32.0,152.0,49.0,9,1.10,9.00,26.00,H


In [332]:
df.loc[df.Result == 'H', 'Result'] = 1
df.loc[df.Result == 'D', 'Result'] = 0
df.loc[df.Result == 'A', 'Result'] = 2

In [333]:
df.head()

,HTW,HTD,HTL,HTGS,HTGC,HTShots,HTShotsTarget,HTOpShots,HTOpShotsTarget,Home,ATW,ATD,ATL,ATGS,ATGC,ATShots,ATShotsTarget,ATOpShots,ATOpShotsTarget,Away,HomeOdd,DrawOdd,AwayOdd,Result
0,3,0,7,7,16,120.0,39.0,138.0,46.0,0,5,4,1,22,10,190.0,64.0,81.0,28.0,1,10.00,5.50,1.29,2
1,3,2,5,8,10,91.0,30.0,124.0,50.0,2,1,2,7,6,15,101.0,32.0,144.0,37.0,3,2.05,3.50,3.50,1
2,3,4,3,8,11,129.0,36.0,143.0,41.0,4,1,3,6,8,16,110.0,29.0,134.0,46.0,5,1.75,3.75,4.50,2
3,3,5,2,10,6,119.0,38.0,107.0,36.0,6,5,1,4,15,15,109.0,31.0,148.0,47.0,7,2.50,3.30,2.80,1
4,6,3,1,17,8,151.0,55.0,86.0,24.0,8,1,3,6,8,20,96.0,32.0,152.0,49.0,9,1.10,9.00,26.00,1


In [334]:
features = df.iloc[:, :-1]
target = df.iloc[:, -1]

In [335]:
features.head()

,HTW,HTD,HTL,HTGS,HTGC,HTShots,HTShotsTarget,HTOpShots,HTOpShotsTarget,Home,ATW,ATD,ATL,ATGS,ATGC,ATShots,ATShotsTarget,ATOpShots,ATOpShotsTarget,Away,HomeOdd,DrawOdd,AwayOdd
0,3,0,7,7,16,120.0,39.0,138.0,46.0,0,5,4,1,22,10,190.0,64.0,81.0,28.0,1,10.00,5.50,1.29
1,3,2,5,8,10,91.0,30.0,124.0,50.0,2,1,2,7,6,15,101.0,32.0,144.0,37.0,3,2.05,3.50,3.50
2,3,4,3,8,11,129.0,36.0,143.0,41.0,4,1,3,6,8,16,110.0,29.0,134.0,46.0,5,1.75,3.75,4.50
3,3,5,2,10,6,119.0,38.0,107.0,36.0,6,5,1,4,15,15,109.0,31.0,148.0,47.0,7,2.50,3.30,2.80
4,6,3,1,17,8,151.0,55.0,86.0,24.0,8,1,3,6,8,20,96.0,32.0,152.0,49.0,9,1.10,9.00,26.00


In [336]:
target.head()

0    2
1    1
2    2
3    1
4    1
Name: Result, dtype: int64

In [337]:
features = features.to_numpy()

In [338]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

features_standardized = scaler.fit_transform(features)

In [339]:
df_not_fact = pd.DataFrame(matches_nn_input, columns=all_cols)

In [340]:
df_not_fact.head()

,HTW,HTD,HTL,HTGS,HTGC,HTShots,HTShotsTarget,HTOpShots,HTOpShotsTarget,Home,ATW,ATD,ATL,ATGS,ATGC,ATShots,ATShotsTarget,ATOpShots,ATOpShotsTarget,Away,HomeOdd,DrawOdd,AwayOdd,Result
0,3,0,7,7,16,120.0,39.0,138.0,46.0,Watford,5,4,1,22,10,190.0,64.0,81.0,28.0,Man City,10.00,5.50,1.29,A
1,3,2,5,8,10,91.0,30.0,124.0,50.0,Swansea,1,2,7,6,15,101.0,32.0,144.0,37.0,West Brom,2.05,3.50,3.50,H
2,3,4,3,8,11,129.0,36.0,143.0,41.0,Southampton,1,3,6,8,16,110.0,29.0,134.0,46.0,Stoke,1.75,3.75,4.50,A
3,3,5,2,10,6,119.0,38.0,107.0,36.0,Man United,5,1,4,15,15,109.0,31.0,148.0,47.0,Crystal Palace,2.50,3.30,2.80,H
4,6,3,1,17,8,151.0,55.0,86.0,24.0,Liverpool,1,3,6,8,20,96.0,32.0,152.0,49.0,Middlesbrough,1.10,9.00,26.00,H


In [354]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_output, test_output = train_test_split(features_standardized,
                                                                      target,
                                                                      test_size=0.15,
                                                                      shuffle=False)

In [342]:
from sklearn.svm import SVC

svc_clf = SVC(C = 1000, kernel = 'rbf', degree = 1, gamma = 'scale')
svc_clf.fit(train_input, train_output)

print(f'SVC score: {svc_clf.score(test_input, test_output)}')

SVC score: 0.42908827785817655


In [355]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(solver='lbfgs', multi_class='auto')
lr_clf.fit(train_input, train_output);
print(f'Logistic regression score: {lr_clf.score(test_input, test_output)}')

Logistic regression score: 0.5294117647058824


In [344]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(abs(train_input), abs(train_output))

print(f'NB score: {nb_clf.score(abs(test_input), abs(test_output))}')

NB score: 0.5021707670043415


In [356]:
from sklearn.neural_network import MLPClassifier

# nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 12, 10, 5, 3, 3, 3, 3, 3, 3), random_state=1)
nn_clf = MLPClassifier(solver='lbfgs',
                       alpha=1e-5,
                       hidden_layer_sizes=(10, 12, 10, 5, 3, 3, 3, 3, 3, 3),
                       random_state=1,
                       activation='relu')
nn_clf.fit(train_input, train_output)

print(f'NN score: {nn_clf.score(test_input, test_output)}')

NN score: 0.532304725168756


In [141]:
# print(nn_clf.n_outputs_)
# print(nn_clf.n_layers_)
# print(nn_clf.coefs_)

# print(train_input.shape)
# print(train_output)

In [358]:
test_pred = []
for i in range(len(test_input)):
    print('#################')
    test_pred.append(nn_clf.predict(test_input[i].reshape(1, -1))[0])
    print(f'Prediction for test case {i}: {nn_clf.predict_proba(test_input[i].reshape(1, -1))[0]} {nn_clf.predict(test_input[i].reshape(1, -1))[0]} --- {test_output.iloc[i]}')


#################
Prediction for test case 0: [0.26340385 0.38065991 0.35593623] 1 --- 0
#################
Prediction for test case 1: [0.25373811 0.3206291  0.42563279] 2 --- 2
#################
Prediction for test case 2: [0.23383994 0.24883545 0.51732461] 2 --- 1
#################
Prediction for test case 3: [0.26713144 0.46943568 0.26343288] 1 --- 0
#################
Prediction for test case 4: [0.26688794 0.47689447 0.25621759] 1 --- 1
#################
Prediction for test case 5: [0.06111779 0.01978765 0.91909456] 2 --- 2
#################
Prediction for test case 6: [0.24120744 0.2715997  0.48719286] 2 --- 2
#################
Prediction for test case 7: [0.24061812 0.6366241  0.12275777] 1 --- 0
#################
Prediction for test case 8: [0.26594278 0.4091543  0.32490293] 1 --- 0
#################
Prediction for test case 9: [0.24061812 0.6366241  0.12275777] 1 --- 2
#################
Prediction for test case 10: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Pr

Prediction for test case 144: [0.24905555 0.30022829 0.45071616] 2 --- 1
#################
Prediction for test case 145: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 146: [0.24959419 0.30242025 0.44798556] 2 --- 0
#################
Prediction for test case 147: [0.26734178 0.44641822 0.28624   ] 1 --- 2
#################
Prediction for test case 148: [0.21582555 0.20336138 0.58081307] 2 --- 2
#################
Prediction for test case 149: [0.26635313 0.41584207 0.3178048 ] 1 --- 2
#################
Prediction for test case 150: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 151: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 152: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 153: [0.25049644 0.30617352 0.44333004] 2 --- 2
#################
Prediction for test case 154: [0.23879139 0.26377524 0.49743337] 2 --- 0
#################

Prediction for test case 236: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 237: [0.13213149 0.07571754 0.79215097] 2 --- 0
#################
Prediction for test case 238: [0.26679604 0.42486586 0.3083381 ] 1 --- 0
#################
Prediction for test case 239: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 240: [0.26560559 0.40436222 0.33003219] 1 --- 2
#################
Prediction for test case 241: [0.26088808 0.36095531 0.37815661] 2 --- 2
#################
Prediction for test case 242: [0.26594858 0.40924158 0.32480984] 1 --- 0
#################
Prediction for test case 243: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 244: [0.26673933 0.42354939 0.30971127] 1 --- 0
#################
Prediction for test case 245: [0.26735853 0.44892028 0.28372119] 1 --- 2
#################
Prediction for test case 246: [0.1382624  0.08231751 0.77942009] 2 --- 2
#################

Prediction for test case 401: [0.15935927 0.10765046 0.73299027] 2 --- 2
#################
Prediction for test case 402: [0.26611787 0.41187014 0.322012  ] 1 --- 1
#################
Prediction for test case 403: [0.26023836 0.3565447  0.38321695] 2 --- 2
#################
Prediction for test case 404: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 405: [0.26687137 0.47732229 0.25580634] 1 --- 1
#################
Prediction for test case 406: [0.26736416 0.45051267 0.28212316] 1 --- 1
#################
Prediction for test case 407: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 408: [0.18034285 0.13756055 0.6820966 ] 2 --- 2
#################
Prediction for test case 409: [0.24061812 0.6366241  0.12275777] 1 --- 0
#################
Prediction for test case 410: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 411: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################

Prediction for test case 563: [0.17753677 0.13324618 0.68921706] 2 --- 2
#################
Prediction for test case 564: [0.24818455 0.29675615 0.4550593 ] 2 --- 1
#################
Prediction for test case 565: [0.26733165 0.44534765 0.2873207 ] 1 --- 1
#################
Prediction for test case 566: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 567: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 568: [0.24061812 0.6366241  0.12275777] 1 --- 2
#################
Prediction for test case 569: [0.26721746 0.46587321 0.26690932] 1 --- 1
#################
Prediction for test case 570: [0.25915236 0.34961228 0.39123536] 2 --- 1
#################
Prediction for test case 571: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 572: [0.26700688 0.47357562 0.2594175 ] 1 --- 1
#################
Prediction for test case 573: [0.23347728 0.24779244 0.51873027] 2 --- 2
#################

Prediction for test case 704: [0.25583255 0.33096383 0.41320362] 2 --- 2
#################
Prediction for test case 705: [0.2672915  0.44219277 0.29051573] 1 --- 1
#################
Prediction for test case 706: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 707: [0.24061812 0.6366241  0.12275777] 1 --- 0
#################
Prediction for test case 708: [0.22322087 0.22064364 0.55613549] 2 --- 0
#################
Prediction for test case 709: [0.26689279 0.47676783 0.25633938] 1 --- 0
#################
Prediction for test case 710: [0.24061812 0.6366241  0.12275777] 1 --- 2
#################
Prediction for test case 711: [0.25094952 0.30809893 0.44095155] 2 --- 1
#################
Prediction for test case 712: [0.20534526 0.18136741 0.61328734] 2 --- 1
#################
Prediction for test case 713: [0.26727333 0.44106006 0.29166661] 1 --- 2
#################
Prediction for test case 714: [0.18485721 0.14472918 0.6704136 ] 2 --- 1
#################

Prediction for test case 864: [0.26702509 0.43101124 0.30196368] 1 --- 1
#################
Prediction for test case 865: [0.24724086 0.29308927 0.45966987] 2 --- 2
#################
Prediction for test case 866: [0.20209891 0.17505361 0.62284749] 2 --- 2
#################
Prediction for test case 867: [0.24061812 0.6366241  0.12275777] 1 --- 2
#################
Prediction for test case 868: [0.26324372 0.37924164 0.35751464] 1 --- 1
#################
Prediction for test case 869: [0.26728243 0.46240363 0.27031394] 1 --- 1
#################
Prediction for test case 870: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 871: [0.25692156 0.33672938 0.40634907] 2 --- 0
#################
Prediction for test case 872: [0.23338395 0.24752504 0.51909101] 2 --- 0
#################
Prediction for test case 873: [0.25848159 0.34556571 0.3959527 ] 2 --- 2
#################
Prediction for test case 874: [0.26711532 0.43400169 0.29888298] 1 --- 0
#################

Prediction for test case 955: [0.26695319 0.47513276 0.25791405] 1 --- 1
#################
Prediction for test case 956: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 957: [0.24061812 0.6366241  0.12275777] 1 --- 0
#################
Prediction for test case 958: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 959: [0.26731554 0.44393253 0.28875193] 1 --- 1
#################
Prediction for test case 960: [0.24061812 0.6366241  0.12275777] 1 --- 1
#################
Prediction for test case 961: [0.26717154 0.46787497 0.26495349] 1 --- 1
#################
Prediction for test case 962: [0.25775459 0.34135514 0.40089027] 2 --- 0
#################
Prediction for test case 963: [0.26009265 0.35558436 0.38432299] 2 --- 0
#################
Prediction for test case 964: [0.26323894 0.37919978 0.35756127] 1 --- 1
#################
Prediction for test case 965: [0.25301802 0.3172721  0.42970988] 2 --- 1
#################

In [376]:
from sklearn.metrics import confusion_matrix
print(len(test_pred))
confusion_matrix(test_output, test_pred, labels=[0, 1, 2])

1037


array([[  0, 197,  80],
       [  0, 410,  66],
       [  0, 142, 142]])